# DAIN Colab

*DAIN Colab, v1.4.0*

Based on the [original Colab file](https://github.com/baowenbo/DAIN/issues/44) by btahir. 

Enhancements by [Styler00Dollar](https://github.com/styler00dollar) aka "sudo rm -rf / --no-preserve-root#8353" on discord and [Alpha](https://github.com/AlphaGit), (Alpha#6137 on Discord). Please do not run this command in your linux terminal. It's rather meant as a joke.

[Styler00Dollar's fork](https://github.com/styler00dollar/DAIN) / [Alpha's fork](https://github.com/AlphaGit/DAIN)


Stuff that should be improved:
- Alpha channel will be removed automatically and won't be added back. Anything related to alpha will be converted to black.
- Adding configuration to select speed
- Detect scenes to avoid interpolating scene-changes
- Auto-resume
- Copy `start_frame` - `end_frame` audio from original input to final output


In [4]:
# Clone DAIN sources
# TODO Bring back original repo
#!git clone https://github.com/baowenbo/DAIN /content/DAIN
!git clone -b colab-compatibility --depth 1 https://github.com/AlphaGit/DAIN

Cloning into 'DAIN'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 142 (delta 42), reused 107 (delta 35), pack-reused 0
Receiving objects: 100% (142/142), 97.54 KiB | 1.52 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [1]:
! rm -R DAIN/frames

rm: cannot remove 'DAIN/frames/input_frames': No such file or directory


In [2]:
! mkdir DAIN/frames
! mkdir DAIN/frames/input_frames && mkdir DAIN/frames/output_frames

In [3]:
################# Required Configurations ############################

#@markdown # Required Configuration
#@markdown Use the values in here to configure what you'd like DAIN to do.

#@markdown ## Input file
#@markdown Path (relative to the root of your Google Drive) to the input file. For instance, if you save your `example.mkv` file in your Google Drive, inside a `videos` folder, the path would be: `videos/example.mkv`. Currenly videos and gifs are supported.
INPUT_FILEPATH = "video/input.mp4" #@param{type:"string"}

#@markdown ## Output file
#@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "video/output.mp4" #@param{type:"string"}

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS = 60 #@param{type:"number"}

#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = 'frames/input_frames' #@param{type:"string"}

#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = 'frames/output_frames' #@param{type:"string"}

#@markdown ## Seamless playback
#@markdown Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = False #@param{type:"boolean"}

#@markdown ## Resize hotfix
#@markdown DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1). Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
#@markdown
#@markdown This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP uses as a fix for this problem, but the original does show such behaviour with the default test images. More advanced users can change the interpolation method. The methods cv2.INTER_CUBIC and cv2.INTER_LANCZOS4 are recommended. The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True #@param{type:"boolean"}

#@markdown ## Auto-remove PNG directory
#@markdown Auto-delete output PNG dir after ffmpeg video creation. Set this to `False` if you want to keep the PNG files.
AUTO_REMOVE = True #@param{type:"boolean"}

In [4]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Quadro P5000, 450.36.06, 16278 MiB


# Install dependencies.

This next step may take somewhere between 15-20 minutes. Run this only once at startup.

Look for the "Finished installing dependencies"  message.

In [26]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]               
Get:4 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [116 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1089 kB]
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 Packages [35.1 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [10.1 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [900 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.

In [31]:
!apt-get install imagemagick imagemagick-doc libgl1-mesa-glx ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  bc fontconfig fontconfig-config fonts-dejavu-core fonts-droid-fallback
  fonts-noto-mono ghostscript gsfonts hicolor-icon-theme imagemagick-6-common
  imagemagick-6-doc imagemagick-6.q16 javascript-common libasyncns0
  libavahi-client3 libavahi-common-data libavahi-common3 libbsd0 libcairo2
  libcap2 libcups2 libcupsfilters1 libcupsimage2 libdatrie1 libdjvulibre-text
  libdjvulibre21 libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2
  libdrm-radeon1 libdrm2 libedit2 libelf1 libfftw3-double3 libflac8
  libfontconfig1 libfontenc1 libfreetype6 libgl1 libgl1-mesa-dri libglapi-mesa
  libglvnd0 libglx-mesa0 libglx0 libgraphite2-3 libgs9 libgs9-common
  libharfbuzz0b libice6 libidn11 libijs-0.35 libilmbase12 libjbig0
  libjbig2dec0 libjpeg-turbo8 libjpeg8 libjs-bootstrap libjs-jquery
  libjs-jquery-easing libjs-jquery-fancybox libjs-jq

In [7]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5MB 13.9MB/s eta 0:00:01
  Found existing installation: pip 19.2.1
    Uninstalling pip-19.2.1:
      Successfully uninstalled pip-19.2.1


In [8]:
# Install known used versions of PyTorch and SciPy
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install scipy==1.1.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [24]:
!pip install opencv-python

     |████████████████████████████████| 49.4 MB 425 kB/s  eta 0:00:01


In [17]:
!pip install imageio

     |████████████████████████████████| 3.3 MB 15.5 MB/s eta 0:00:01


In [6]:
import os
import cv2

In [12]:
%cd notebooks

/notebooks


In [13]:
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd DAIN/my_package/
!./build.sh
print("Building #1 done.")
%cd ../../

/notebooks/DAIN/my_package
Need pytorch>=1.0.0
./build.sh: line 4: activate: No such file or directory
running install
running bdist_egg
running egg_info
creating mindepthflowprojection_cuda.egg-info
writing mindepthflowprojection_cuda.egg-info/PKG-INFO
writing dependency_links to mindepthflowprojection_cuda.egg-info/dependency_links.txt
writing top-level names to mindepthflowprojection_cuda.egg-info/top_level.txt
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
reading manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'mindepthflowprojection_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/lo

In [19]:
%cd notebooks

/notebooks


In [20]:
# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")
%cd ../../..

/notebooks/DAIN/PWCNet/correlation_package_pytorch1_0
Need pytorch>=1.0.0
./build.sh: line 4: activate: No such file or directory
running install
running bdist_egg
running egg_info
creating correlation_cuda.egg-info
writing correlation_cuda.egg-info/PKG-INFO
writing dependency_links to correlation_cuda.egg-info/dependency_links.txt
writing top-level names to correlation_cuda.egg-info/top_level.txt
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
reading manifest file 'correlation_cuda.egg-info/SOURCES.txt'
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'correlation_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/li

In [22]:
# Downloading pre-trained model
!mkdir DAIN/model_weights
!wget -O DAIN/model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

--2020-09-14 20:22:33--  http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
Resolving vllab1.ucmerced.edu (vllab1.ucmerced.edu)... 169.236.184.68
Connecting to vllab1.ucmerced.edu (vllab1.ucmerced.edu)|169.236.184.68|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96319643 (92M)
Saving to: ‘DAIN/model_weights/best.pth’

DAIN/model_weights/ 100%[===================>]  91.86M  21.8MB/s    in 3.8s    

2020-09-14 20:22:37 (24.3 MB/s) - ‘DAIN/model_weights/best.pth’ saved [96319643/96319643]



In [4]:
!CUDA_VISIBLE_DEVICES=0
print("Finished installing dependencies.")

Finished installing dependencies.


In [7]:
# Detecting FPS of input file.

filename = os.path.basename(INPUT_FILEPATH)

cap = cv2.VideoCapture(f'video/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

In [8]:
# ffmpeg extract - Generating individual frame PNGs from the source file.
! ffmpeg -i 'video/{filename}' 'DAIN/{FRAME_INPUT_DIR}/%05d.png'

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [9]:
! ls 'DAIN/{FRAME_INPUT_DIR}' | wc -l

1625


In [10]:
%cd DAIN/
from IPython.display import clear_output
clear_output()

pngs_generated_count = 1625

import shutil
if SEAMLESS==True:
  pngs_generated_count += 1
  original = str(FRAME_INPUT_DIR)+"/00001.png"
  target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)

print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

# Checking if PNG do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

%cd ../../

Input FPS: 23.98
1625 frame PNGs generated.
/notebooks/DAIN/frames/input_frames
srgb detected
/notebooks/DAIN


In [11]:
# Interpolation

In [ ]:
!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

revise the unique id to a random numer 72979
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/72979-Tue-Sep-15-19-49/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, end_frame=1625, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, frame_input_dir='frames/input_frames', frame_output_dir='frames/output_frames', log='./model_weights/72979-Tue-Sep-15-19-49/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/72979-Tue-Sep-15-19-49', save_which=1, seed=1, start_frame=1, time_step=0.39966666666666667, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 1 frames
****** Processed frame 1 | Time per frame (avg): 11.03s | Time left: 4:58:24.816595 ***************

In [ ]:
# Finding DAIN Frames, upscaling and cropping to match original
import numpy as np
if(RESIZE_HOTFIX==True):
  images = []
  for filename in os.listdir(f'{FRAME_OUTPUT_DIR}'):
    img = cv2.imread(os.path.join(f'{FRAME_OUTPUT_DIR}',filename))
    part_filename = os.path.splitext(filename)
    if(part_filename[0].endswith('0')==False):
      dimension = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      ns = str(f'{FRAME_OUTPUT_DIR}/')+part_filename[0]+".png"
      print(ns)
      cv2.imwrite(ns, crop)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def img2video(read_path,save_path):
    print(read_path,save_path)
    files = []
    for r, _, f in os.walk(read_path):
        for file in f:
            if '.png' in file:
                files.append([os.path.join(r, file),file.replace(".png","")])
    print("")
    files.sort(key=lambda x: int(x[1]))
    img = cv2.imread(files[0][0])
    height, width, _ = img.shape
    size = (width,height)
    print("VidSize: ",str(size))
    print("")
    #-----------------------------------------------
    out = cv2.VideoWriter(save_path+"reconstucted.mp4",cv2.VideoWriter_fourcc(*'mp4v'), 60, size)

    for file in tqdm(range(len(files))):
        path = files[file][0]
        img = cv2.imread(path)
        out.write(img)
    out.release()
    

In [ ]:
!ls

In [ ]:
img2video(FRAME_OUTPUT_DIR,'../'+str(OUTPUT_FILE_PATH))